In [4]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import chart_studio.plotly as py
import chart_studio
import matplotlib.pyplot as plt

chart_studio.tools.set_credentials_file(username='oystein.nerva', api_key='0ODcZ6UisJ8n0AjEQt9N')

In [36]:
# finn API url her: https://data.ssb.no/api/v0/dataset

url = "https://data.ssb.no/api/v0/dataset/1078.csv?lang=no"

df = pd.read_csv(url, sep=";", decimal=",", encoding="ISO-8859-1")
df.head()

,region,kjønn,alder,år,statistikkvariabel,"07459: Befolkning, etter region, kjønn, alder, år og statistikkvariabel"
0,3001 Halden,1 Menn,F00-04 0-4 år,2020,Personer,761
1,3001 Halden,1 Menn,F05-09 5-9 år,2020,Personer,888
2,3001 Halden,1 Menn,F10-14 10-14 år,2020,Personer,947
3,3001 Halden,1 Menn,F15-19 15-19 år,2020,Personer,1029
4,3001 Halden,1 Menn,F20-24 20-24 år,2020,Personer,1020


In [37]:
df["rest"], df["age"], df["rest2"] = df["alder"].str.split(" ",2).str
df = df.drop(columns = ["rest", "rest2", "statistikkvariabel", "år", "alder"])

df["aldersgruppe"], df["rest"] = df["age"].str.split("-",1).str
df["aldersgruppe"] = pd.to_numeric(df["aldersgruppe"], errors='coerce')
df = df.drop(columns = ["rest", "age"])

df["Kommunenr"], df["Kommune"] = df["region"].str.split(" ",1).str
df = df.drop(columns="region")

df.head()

,kjønn,"07459: Befolkning, etter region, kjønn, alder, år og statistikkvariabel",aldersgruppe,Kommunenr,Kommune
0,1 Menn,761,0,3001,Halden
1,1 Menn,888,5,3001,Halden
2,1 Menn,947,10,3001,Halden
3,1 Menn,1029,15,3001,Halden
4,1 Menn,1020,20,3001,Halden


In [38]:
df = df.groupby(["Kommunenr", "Kommune", "aldersgruppe"]).sum().reset_index()
df["aldersgruppe"] = df.apply(lambda x: "over 65" if x["aldersgruppe"] >= 65 else "under 65", axis=1)
df = df.groupby(["Kommunenr", "Kommune", "aldersgruppe"]).sum().reset_index()

df.head()

,Kommunenr,Kommune,aldersgruppe,"07459: Befolkning, etter region, kjønn, alder, år og statistikkvariabel"
0,0301,Oslo kommune,over 65,87380
1,0301,Oslo kommune,under 65,606114
2,1101,Eigersund,over 65,2687
3,1101,Eigersund,under 65,12124
4,1103,Stavanger,over 65,20729


In [41]:
df = df.pivot_table(index=["Kommunenr", "Kommune"], columns=["aldersgruppe"], values="07459: Befolkning, etter region, kjønn, alder, år og statistikkvariabel").reset_index()
df["andel over 65"] = (df["over 65"]/(df["over 65"]+df["under 65"]))*100
df.head()

aldersgruppe,Kommunenr,Kommune,over 65,under 65,andel over 65
0,0301,Oslo kommune,87380,606114,12.599965
1,1101,Eigersund,2687,12124,18.141922
2,1103,Stavanger,20729,122845,14.437851
3,1106,Haugesund,6342,31015,16.976738
4,1108,Sandnes,10292,69245,12.939890


# tettbebygd strøk

In [106]:
bebyggelse = pd.read_excel("Folkemengde-2.xlsx")
bebyggelse["Kommunenr"], bebyggelse["Kommune"] = bebyggelse["Kommune"].str.split(" ",1).str
bebyggelse["rest"], bebyggelse["Kommunenr"] = bebyggelse["Kommunenr"].str.split("-",1).str
bebyggelse = bebyggelse.drop(columns = ["rest"])

innbyggere = bebyggelse.groupby(["Kommunenr"])["Befolkning"].sum().reset_index()
bebyggelse.head()

,Kommune,Type,Befolkning,Kommunenr
0,Halden,Tettbygd strøk,27084,3001
1,Halden,Spredtbygd strøk,4043,3001
2,Halden,Ukjent,50,3001
3,Moss,Tettbygd strøk,46901,3002
4,Moss,Spredtbygd strøk,1863,3002


In [107]:
bebyggelse = bebyggelse.pivot_table(index=["Kommunenr", "Kommune"], columns=["Type"], values="Befolkning").reset_index()
bebyggelse["Andel i tettebebyggelse"] = ((bebyggelse["Tettbygd strøk"]/(bebyggelse["Tettbygd strøk"]+bebyggelse["Spredtbygd strøk"]+bebyggelse["Ukjent"])))*100
bebyggelse = bebyggelse.drop(columns = ["Kommune", "Spredtbygd strøk", "Tettbygd strøk", "Ukjent"])
bebyggelse = bebyggelse.merge(innbyggere, how="left", on="Kommunenr")
bebyggelse.head()

,Kommunenr,Andel i tettebebyggelse,Befolkning
0,0301,99.422674,681071
1,1101,82.805125,14830
2,1103,95.848881,142034
3,1106,97.535570,37250
4,1108,93.825775,78439


# tilhørighet/mapping

In [159]:
kom = pd.read_csv("kommunetilhorighet_2020.csv", sep=";", decimal=",")
kom.tail()

,Fylkesnummer,Fylke,Kommunenummer,Kommune
351,54,Troms og Finnmark - Romsa ja Finnmárku - Troms...,5440,Berlevåg
352,54,Troms og Finnmark - Romsa ja Finnmárku - Troms...,5441,Deatnu - Tana
353,54,Troms og Finnmark - Romsa ja Finnmárku - Troms...,5442,Unjárga - Nesseby
354,54,Troms og Finnmark - Romsa ja Finnmárku - Troms...,5443,Båtsfjord
355,54,Troms og Finnmark - Romsa ja Finnmárku - Troms...,5444,Sør-Varanger


In [160]:
kom["Fylke"], kom["rest"] = kom["Fylke"].str.split("-",1).str
kom.tail()

,Fylkesnummer,Fylke,Kommunenummer,Kommune,rest
351,54,Troms og Finnmark,5440,Berlevåg,Romsa ja Finnmárku - Tromssa ja Finmarkku
352,54,Troms og Finnmark,5441,Deatnu - Tana,Romsa ja Finnmárku - Tromssa ja Finmarkku
353,54,Troms og Finnmark,5442,Unjárga - Nesseby,Romsa ja Finnmárku - Tromssa ja Finmarkku
354,54,Troms og Finnmark,5443,Båtsfjord,Romsa ja Finnmárku - Tromssa ja Finmarkku
355,54,Troms og Finnmark,5444,Sør-Varanger,Romsa ja Finnmárku - Tromssa ja Finmarkku


# JOIN

In [161]:
data = df.merge(bebyggelse, how="left", on="Kommunenr")
data.head()

,Kommunenr,Kommune,over 65,under 65,andel over 65,Andel i tettebebyggelse,Befolkning
0,0301,Oslo kommune,87380,606114,12.599965,99.422674,681071
1,1101,Eigersund,2687,12124,18.141922,82.805125,14830
2,1103,Stavanger,20729,122845,14.437851,95.848881,142034
3,1106,Haugesund,6342,31015,16.976738,97.535570,37250
4,1108,Sandnes,10292,69245,12.939890,93.825775,78439


In [162]:
data["Kommunenr"] = pd.to_numeric(data["Kommunenr"], errors='coerce')
data = data.merge(kom, how="left", left_on="Kommunenr", right_on="Kommunenummer")


In [109]:
results, bin_edges = pd.qcut(data['Andel i tettebebyggelse'], q=3, labels=["Lav", "Mid", "Høy"], retbins=True)
bebyggelse_thresh = pd.DataFrame(zip(bin_edges, ["Lav", "Mid", "Høy"]), columns=['Threshold', 'Tekst'])
bebyggelse_thresh

,Threshold,Tekst
0,0.000000,Lav
1,44.550362,Mid
2,69.745587,Høy


In [110]:
results, bin_edges = pd.qcut(data['andel over 65'], q=3, labels=["Lav", "Mid", "Høy"], retbins=True)
alder_thresh = pd.DataFrame(zip(bin_edges, ["Lav", "Mid", "Høy"]), columns=['Threshold', 'Tekst'])
alder_thresh

,Threshold,Tekst
0,11.648059,Lav
1,20.087025,Mid
2,23.191483,Høy


In [189]:
legend = [i for i in data["Fylke"].dropna().unique()]
legend.sort()

grey = "rgba(192,192,192,1)"
b = "rgb(9,28,87)"
maroon = "rgb(200,33,72)"

fig = px.scatter(data, 
                 x="Andel i tettebebyggelse", 
                 y="andel over 65", 
                 category_orders={"Fylke": legend},
                 color="Fylke",
                 hover_name="Kommune_x",
                 size="Befolkning",
                 size_max=60,
                 #marginal_x="histogram", marginal_y="histogram",
                 template = "plotly_white",
                 range_x=[-1,110], range_y=[8,35],
                 title = "<b>Potensielt sårbare kommuner i Norge</b><br>Alderssammensetning og tettebebyggelse farget etter tertiler.<br>Datakilde: ssb.no",
                 labels = {"level_1": "Dager siden antall smittede passerte 5", "variable": "Dato", "value": "Totalt registrerte smittede"}    
                )

fig.add_shape(type="rect",
            x0=-1, y0=8,
            x1=110, y1=20.087025,
            line_width=0,layer="below",
            fillcolor=maroon, opacity=0.1,)

fig.add_shape(type="rect",
            x0=-1, y0=20.087025,
            x1=110, y1=23.191483,
            line_width=0,layer="below",
            fillcolor=maroon, opacity=0.4)

fig.add_shape(type="rect",
            x0=-1, y0=23.191483,
            x1=110, y1=35,
            line_width=0,layer="below",
            fillcolor=maroon, opacity=0.7)

#vertical areas:
fig.add_shape(type="rect",
            x0=-1, y0=8,
            x1=44.550362, y1=35,
            line_width=0,layer="below",
            fillcolor=b, opacity=0.1,)

fig.add_shape(type="rect",
            x0=44.550362, y0=8,
            x1=69.745587, y1=35,
            line_width=0,layer="below",
            fillcolor=b, opacity=0.35)

fig.add_shape(type="rect",
            x0=69.745587, y0=8,
            x1=110, y1=35,
            line_width=0,layer="below",
            fillcolor=b, opacity=0.7)

fig.update_layout(
    updatemenus = list([
    dict(type="buttons", 
         direction="left", 
         active=0,
         buttons=list([
                        dict(label='Hele landet',
                 method='update',
                 args=[{'visible': [True, True, True, True, True, True, True, True, True, True, True]}]), 
            dict(label='Midt',
                 method='update',
                 args=[{'visible': [False, False, True, False, False, False, False, True, False, False, False]}]),         
            dict(label='Nord',
                 method='update',
                 args=[{'visible': [False, False, False, True, False, False, True, False, False, False, False]}]),
            
            dict(label='Sør-Øst',
                 method='update',
                 args=[{'visible': [True, True, False, False, True, False, False, False, True, False, True]}]),  
                               
             dict(label='Vest',
                 method='update',
                 args=[{'visible': [False, False, False, False, False, True, False, False, False, True, False]},
                       ])  
             
            ]),
            showactive=True,
            x=0.5,
            xanchor="center",
            y=-0.2,
            yanchor="top"
        )
    ]))

fig.update_layout(autosize=True,
    margin=dict(l=100, r=100, t=120, b=100),
    paper_bgcolor='rgba(0, 0, 0, 0)',
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    #plot_bgcolor='rgb(248, 248, 255),'
                 )

#py.plot(fig, filename='sarbare_kommuner.html', auto_open=False)
fig.show()

In [194]:
data["risk_score"] = (data["andel over 65"] - data["andel over 65"].min())/(data["andel over 65"].max()-data["andel over 65"].min()) * (data["Andel i tettebebyggelse"] - data["Andel i tettebebyggelse"].min())/(data["Andel i tettebebyggelse"].max()-data["Andel i tettebebyggelse"].min())
data.head()

,Kommunenr,Kommune_x,over 65,under 65,andel over 65,Andel i tettebebyggelse,Befolkning,Fylkesnummer,Fylke,Kommunenummer,Kommune_y,rest,risk_score
0,301,Oslo kommune,87380,606114,12.599965,99.422674,681071,3,Oslo,301,Oslo,NaN,0.047374
1,1101,Eigersund,2687,12124,18.141922,82.805125,14830,11,Rogaland,1101,Eigersund,NaN,0.269168
2,1103,Stavanger,20729,122845,14.437851,95.848881,142034,11,Rogaland,1103,Stavanger,NaN,0.133851
3,1106,Haugesund,6342,31015,16.976738,97.535570,37250,11,Rogaland,1106,Haugesund,NaN,0.260163
4,1108,Sandnes,10292,69245,12.939890,93.825775,78439,11,Rogaland,1108,Sandnes,NaN,0.060672


In [199]:
fig = px.bar(data, 
             x='risk_score', y='Fylke', 
             orientation='h',
             category_orders={"Fylke": legend},
             height=500,
             title = "<b>Fylker sortert etter potensiell sårbarhet</b><br>Datakilde: ssb.no.",
             color='Fylke',
             #hover_name="Fylke",
             #hover_data=["Andel over 65 år"],
             template = "plotly_white", 
             #labels = {"Vekstrate": "daglig vekst (%)", "- 1 uke": "Daglig vekst for en uke siden", "text": "Dager før dobling med dagens vekst"},
            )


fig.layout.coloraxis.showscale = False

fig.update_layout(
    yaxis=dict(
        separatethousands=True,
        categoryorder = 'total ascending',
        title_text="",
        fixedrange=True,
    ), xaxis=dict(fixedrange=True,
                 title_text=""),
    title_font=dict(family="Helvetica", size=18),
        margin=dict(b=150)) #bottom margin

fig.update_layout(autosize=True,
    margin=dict(l=100, r=100, t=150, b=0),
    paper_bgcolor='rgba(0, 0, 0, 0)'
    #plot_bgcolor='rgb(248, 248, 255)'    
                 )

#plotly.offline.plot(fig, filename='COVID_moment_fylke.html')
#py.plot(fig, filename='COVID19_vekstmomentum_fylker.html', auto_open=False)
fig.show()